In [57]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
import copy
import matplotlib.pyplot as plt
from yahooquery import Ticker

In [58]:
#importando base
df_ibrx = pd.read_excel('/Users/user/Desktop/Empresa/ETFs_Onshore.xlsx', sheet_name = 'IBRX100')
df_ibrx.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Setor,Subsetor,Segmento
0,ABEV3,AMBEV S/A,ON,4.354.228.928,2936.0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
1,ALPA4,ALPARGATAS,PN N1,187.007.704,328.0,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
2,ALSO3,ALIANSCSONAE,ON NM,127.374.163,186.0,Financeiro,Exploração de Imóveis,Exploração de Imóveis
3,AZUL4,AZUL,PN N2,317.471.474,363.0,Bens Industriais,Transporte,Transporte Aéreo
4,B3SA3,B3,ON NM,2.046.021.644,6301.0,Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos


In [59]:
# dando aquele tapa
df = df_ibrx
df = pd.DataFrame(df) 
df = df.dropna()
df['Part. (%)'] /= 100000
df.rename(columns={'Código\xa0':'Codigo'}, inplace = True)
df['Codigo'] = df['Codigo'] + ".SA"
tickers = df['Codigo']

In [60]:
start = dt.date.today() + dt.timedelta(1) - dt.timedelta(59)
end = dt.date.today() + dt.timedelta(1)
prices = {}
for t in tickers:
    try:
        prices[t] = yf.download(t,start,end,interval='1d')  
    except: 
        pass
    prices[t].dropna(how='all', inplace = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [61]:
prices2 = copy.deepcopy(prices)
for t in tickers:
    prices2[t]['roll_avg_5d'] = prices2[t]['Close'].rolling(5).mean()
    prices2[t]['roll_avg_20d'] = prices2[t]['Close'].rolling(20).mean()
    prices2[t]['roll_max_5d'] = prices2[t]['Close'].rolling(5).max()
    prices2[t]['roll_max_20d'] = prices2[t]['Close'].rolling(20).max()
    prices2[t]['roll_min_5d'] = prices2[t]['Close'].rolling(5).min()
    prices2[t]['roll_min_20d'] = prices2[t]['Close'].rolling(20).min()
    prices2[t].dropna(inplace=True)

In [62]:
prices3 = pd.DataFrame(columns=['ticker','Setor', 'Subsetor', 'Segmento', 'Close', 'Close_D-1', 'roll_avg_5d', 'roll_avg_20d',
                                'roll_max_5d', 'roll_max_20d', 'roll_min_5d', 'roll_min_20d', 'Delta_1d', 'Delta_5d',
                               'Delta_20d', 'Max_desconto', 'Ranking_1','Ranking_2','Ranking_3','Ranking_4','Soma_total',
                               'Ranking_Final'])
for t in range(len(tickers)):
    prices3.loc[t,'ticker'] = str(tickers[t]).replace('.SA','')
    prices3.iloc[t,1] = df[df['Codigo'] == tickers[t]]['Setor'].to_list()[0]
    prices3.iloc[t,2] = df[df['Codigo'] == tickers[t]]['Subsetor'].to_list()[0]
    prices3.iloc[t,3] = df[df['Codigo'] == tickers[t]]['Segmento'].to_list()[0]
    prices3.iloc[t,4] = prices2[tickers[t]]['Close'][-1]
    prices3.iloc[t,5] = prices2[tickers[t]]['Close'][-2]
    prices3.iloc[t,6] = prices2[tickers[t]]['roll_avg_5d'][-1]
    prices3.iloc[t,7] = prices2[tickers[t]]['roll_avg_20d'][-1]
    prices3.iloc[t,8] = prices2[tickers[t]]['roll_max_5d'][-1]
    prices3.iloc[t,9] = prices2[tickers[t]]['roll_max_20d'][-1]
    prices3.iloc[t,10] = prices2[tickers[t]]['roll_min_5d'][-1]
    prices3.iloc[t,11] = prices2[tickers[t]]['roll_min_20d'][-1]
    prices3.iloc[t,12] = prices3.iloc[t,4]/prices3.iloc[t,5]-1 #Delta_1d
    prices3.iloc[t,13] = prices3.iloc[t,4]/prices3.iloc[t,6]-1 #Delta_5d
    prices3.iloc[t,14] = prices3.iloc[t,4]/prices3.iloc[t,7]-1 #Delta_20d
    prices3.iloc[t,15] = prices3.iloc[t,4]/prices3.iloc[t,9]-1 #Max_desconto
    prices3['Ranking_1'] = prices3['Delta_1d'].rank(ascending= True)
    prices3['Ranking_2'] = prices3['Delta_5d'].rank(ascending= False)
    prices3['Ranking_3'] = prices3['Delta_20d'].rank(ascending= True)
    prices3['Ranking_4'] = prices3['Max_desconto'].rank(ascending= True)
    prices3['Soma_total'] = prices3['Ranking_1'] + prices3['Ranking_2'] + prices3['Ranking_3'] + prices3['Ranking_4']
    prices3['Ranking_Final'] = prices3['Soma_total'].rank(ascending= True)

prices3.sort_values(['Ranking_Final'], ascending = True)    

,ticker,Setor,Subsetor,Segmento,Close,Close_D-1,roll_avg_5d,roll_avg_20d,roll_max_5d,roll_max_20d,...,Delta_1d,Delta_5d,Delta_20d,Max_desconto,Ranking_1,Ranking_2,Ranking_3,Ranking_4,Soma_total,Ranking_Final
32,CYRE3,Consumo Cíclico,Construção Civil,Incorporações,24.27,24.66,24.248,25.3315,24.66,27.62,...,-0.0158151,0.000907312,-0.0419043,-0.121289,14.0,52.0,15.0,13.0,94.0,1.0
60,LIGT3,Utilidade Pública,Energia Elétrica,Energia Elétrica,15.63,16.19,15.9,16.976,16.19,18.92,...,-0.0345893,-0.0169811,-0.0792884,-0.17389,3.0,91.0,4.0,3.0,101.0,2.5
51,HYPE3,Saúde,Comércio e Distribuição,Medicamentos e Outros Produtos,32.39,32.68,32.308,34.1985,32.88,36.3,...,-0.00887396,0.00253805,-0.0528824,-0.107713,28.0,47.0,9.0,17.0,101.0,2.5
41,ENGI11,Utilidade Pública,Energia Elétrica,Energia Elétrica,44.55,45.23,44.672,47.1205,45.24,50.25,...,-0.0150343,-0.00273104,-0.0545516,-0.113433,15.0,65.0,8.0,14.0,102.0,4.0
17,BRML3,Financeiro,Exploração de Imóveis,Exploração de Imóveis,9.71,9.83,9.746,10.1555,9.88,11.32,...,-0.0122075,-0.00369383,-0.0438679,-0.142226,18.0,68.0,12.0,7.0,105.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,VALE3,Materiais Básicos,Mineração,Minerais Metálicos,62.62,62.2,62.644,61.5915,63.06,63.06,...,0.00675238,-0.000383138,0.0166987,-0.00697752,71.0,59.0,73.0,95.0,298.0,96.0
64,MGLU3,Consumo Cíclico,Comércio,Eletrodomésticos,87.07,87.05,88.134,83.5875,89.7,89.7,...,0.000229714,-0.0120725,0.0416629,-0.0293199,51.0,86.0,85.0,78.0,300.0,97.0
47,GOAU4,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia,9.06,9.04,9.098,8.457,9.28,9.28,...,0.00221244,-0.00417669,0.0713019,-0.0237068,58.0,71.0,91.0,81.0,301.0,98.0
94,USIM5,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia,9.68,9.57,9.62,8.7205,9.75,9.75,...,0.0114943,0.00623705,0.110028,-0.00717946,77.0,40.0,97.0,94.0,308.0,99.0


In [63]:
prices3.to_excel('/Users/user/Desktop/Empresa/IBRX100.xlsx',sheet_name = 'IBRX100')